In [16]:
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from tqdm import tqdm
import random

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

In [27]:
def makeUrl(search, start_date, end_date, start_pg=1, end_pg=294):
    date_part = "&pd=3&ds=" + start_date + "&de=" + end_date
    return ["https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + date_part + "&sort=2&start=" + str((i - 1) * 10 + 1) for i in range(start_pg, end_pg + 1)]

In [28]:
def get_naver_articles(urls):
    articles, titles, dates, press = [], [], [], []
    
    for url in urls:
        html = requests.get(url, headers=headers)
        soup = BeautifulSoup(html.text, "html.parser")

        article_links = [link.attrs['href'] for link in soup.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")]
        article_titles = [title.text for title in soup.select(".news_tit")]
        article_infos = [info.text for info in soup.select(".info")]

        new_urls = []
        idx = 0

        while idx < len(article_links) - 1:
            if 'news.naver.com' in article_links[idx]:
                # Check if the next link also contains 'news.naver.com'
                if idx + 1 < len(article_links) and 'news.naver.com' in article_links[idx + 1]:
                    new_urls.append(article_links[idx + 1])
                    idx += 2
                else:
                    new_urls.append(article_links[idx])
                    idx += 1
            elif 'news.naver.com' in article_links[idx + 1]:
                new_urls.append(article_links[idx + 1])
                idx += 2
            else:
                new_urls.append(article_links[idx])
                idx += 1

        # 마지막 요소 처리
        if idx == len(article_links) - 1:
            new_urls.append(article_links[idx])

        date_pattern = re.compile(r"\d{4}\.\d{2}\.\d{2}")

        current_dates = []
        current_press = []

        for idx, info in enumerate(article_infos):
            match = date_pattern.search(info)
            if match:
                current_dates.append(match.group(0))
                
                if idx > 0:  # 인덱스 0 앞에는 데이터가 없으므로 idx > 0 조건 추가
                    current_press.append(article_infos[idx - 1])

        articles.extend(new_urls)
        titles.extend(article_titles)
        dates.extend(current_dates)
        press.extend(current_press)

        time.sleep(random.uniform(0.8, 1.2))
        
    return articles, titles, dates, press

In [29]:
def get_article_contents(article_urls):
    contents = []
    article_time = []  # 이름 변경
    
    for url in tqdm(article_urls):
        if 'news.naver.com' in url:
            news = requests.get(url, headers=headers)
            soup = BeautifulSoup(news.text, "html.parser")

            content = soup.select("#dic_area") or soup.select("#articeBody")
            cleaned_content = re.sub('<[^>]*>', '', ''.join(str(item) for item in content)).replace("flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", '')
            contents.append(cleaned_content)

            try:
                date = soup.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span").attrs['data-date-time']
            except AttributeError:
                date = re.sub('<[^>]*>', '', str(soup.select_one("#content > div.end_ct > div > div.article_info > span > em")))
            article_time.append(date)  # 이름 변경
        else:
            contents.append('')
            article_time.append('')
    
    return contents, article_time


In [30]:
# def main():
#     search = input("검색할 키워드를 입력해주세요:")
#     start_date = input("\n크롤링할 시작 날짜를 입력해주세요. ex)2022.01.01:")
#     end_date = input("\n크롤링할 종료 날짜를 입력해주세요. ex)2022.12.31:")

#     urls = makeUrl(search, start_date, end_date)
#     article_urls, article_titles, article_dates, article_press = get_naver_articles(urls)
    
#     contents, article_time = get_article_contents(article_urls)  # 이름 변경
    
#     df = pd.DataFrame({'date': article_dates, 'time': article_time, 'title': article_titles, 'content': contents,'press': article_press, 'link': article_urls })  # 이름 변경
#     df.drop_duplicates(keep='first', inplace=True, ignore_index=True)
#     return df

# if __name__ == "__main__":
#     result_df = main()
#     print(result_df)

In [31]:
search = input("검색할 키워드를 입력해주세요:")
start_date = input("\n크롤링할 시작 날짜를 입력해주세요. ex)2022.01.01:")
end_date = input("\n크롤링할 종료 날짜를 입력해주세요. ex)2022.12.31:")

urls = makeUrl(search, start_date, end_date)
article_urls, article_titles, article_dates, article_press = get_naver_articles(urls)

contents, article_time = get_article_contents(article_urls)  # 이름 변경

# 여기서 각 리스트의 길이를 확인합니다.
print("Length of article_urls:", len(article_urls))
print("Length of article_titles:", len(article_titles))
print("Length of article_dates:", len(article_dates))
print("Length of article_press:", len(article_press))
print("Length of contents:", len(contents))
print("Length of article_time:", len(article_time))

# 길이가 동일한지 확인
if len(article_titles) == len(article_dates)== len(article_press) == len(article_urls) == len(contents):
    df = pd.DataFrame({'date': article_dates, 'time': article_time, 'title': article_titles, 'content': contents,'press': article_press, 'link': article_urls }) 
    df.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    print(df)
else:
    print("The lengths of the lists are not equal. Dataframe cannot be created.")



100%|██████████| 2940/2940 [08:44<00:00,  5.61it/s]

Length of article_urls: 2940
Length of article_titles: 2940
Length of article_dates: 2940
Length of article_press: 2940
Length of contents: 2940
Length of article_time: 2940
            date                 time  \
0     2018.11.23                        
1     2018.11.23  2018-11-23 00:01:10   
2     2018.11.23                        
3     2018.11.23                        
4     2018.11.23                        
...          ...                  ...   
2923  2018.11.30  2018-11-30 23:31:17   
2924  2018.11.30                        
2925  2018.11.30                        
2926  2018.11.30                        
2927  2018.11.30                        

                                                  title  \
0     비트코인 0.49%↑ 이더리움 0.84%↑ 리플 0.40%↑ 대시 0.89%↑ 에이...   
1     [가상화폐 뉴스] 11월 23일 00시 00분 비트코인(1.57%), 아이오타(7....   
2     비트코인 0.47%↑ 이더리움 0.67%↑ 리플 0.20%↑ 퀀텀 1.83%↑ 라이...   
3     비트코인 0.45%↑ 이더리움 0.67%↑ 리플 0.40%↑ 라이트코인 0.78%↑...   
4         [암호화폐 시황] 비트코인 0.47%↑·이더리움 0

In [32]:
df

,date,time,title,content,press,link
0,2018.11.23,,비트코인 0.49%↑ 이더리움 0.84%↑ 리플 0.40%↑ 대시 0.89%↑ 에이...,,로이슈,http://www.lawissue.co.kr
1,2018.11.23,2018-11-23 00:01:10,"[가상화폐 뉴스] 11월 23일 00시 00분 비트코인(1.57%), 아이오타(7....",\n[한국경제TV 라이온봇 기자]\n\n\n\n[그림 1] 가상화폐 시세 (제공: ...,한국경제TV,https://n.news.naver.com/mnews/article/215/000...
2,2018.11.23,,비트코인 0.47%↑ 이더리움 0.67%↑ 리플 0.20%↑ 퀀텀 1.83%↑ 라이...,,로이슈,http://www.lawissue.co.kr
3,2018.11.23,,비트코인 0.45%↑ 이더리움 0.67%↑ 리플 0.40%↑ 라이트코인 0.78%↑...,,로이슈,http://www.lawissue.co.kr
4,2018.11.23,,[암호화폐 시황] 비트코인 0.47%↑·이더리움 0.67%↑...0시 00분 현재,,글로벌경제,http://www.getnews.co.kr
...,...,...,...,...,...,...
2923,2018.11.30,2018-11-30 23:31:17,"[가상화폐 뉴스] 11월 30일 23시 30분 비트코인(-6.28%), 이오스(-1...",\n[한국경제TV 라이온봇 기자]\n\n\n\n[그림 1] 가상화폐 시세 (제공: ...,한국경제TV,https://n.news.naver.com/mnews/article/215/000...
2924,2018.11.30,,[암호화폐 시황] 비트코인 0.51%↑ 비트코인캐시 1.04%↑ 비트코인골드 0% ...,,빅데이터뉴스,http://www.thebigdata.co.kr/
2925,2018.11.30,,"[세계 거래소별 가상화폐 시황] 30일, 23시 30분 현재",,로이슈,http://www.lawissue.co.kr
2926,2018.11.30,,"[암호화폐 시황] 코인제스트, 23시 40분 , 비트코인(0.53%), 이더리움(0...",,로이슈,http://www.lawissue.co.kr


In [79]:
# end_date = '2017.10.09'

In [102]:
#데이터 프레임 저장
filename = '{}_{}_{}.csv'.format(search, start_date.replace(".", ""), end_date.replace(".", ""))
df.to_csv(filename, encoding='utf-8-sig', index=False)

In [55]:
article_urls[70:80]

['http://sports.news.naver.com/column/columnList.nhn?expertId=437',
 'https://sports.news.naver.com/news.nhn?oid=260&aid=0000001176',
 'https://n.news.naver.com/mnews/article/008/0003920735?sid=101',
 'https://n.news.naver.com/mnews/article/374/0000137905?sid=101',
 'http://www.digitaltoday.co.kr/',
 'http://www.etoday.co.kr',
 'http://www.etoday.co.kr',
 'http://www.getnews.co.kr',
 'http://www.koreatimes.com',
 'http://www.getnews.co.kr']

In [27]:
article_titles[3990:4000]

['[가상화폐 시황] 비트코인 -0.29%↓·이더리움 0.67%↑...17시 00분 현재',
 '멕시코 중앙은행 총재 “보증없는 비트코인은 화폐 아니다”',
 '비트코인 억만장자 등극한 스웨덴 개발자…비결은?',
 '[가상화폐 시황] 비트코인 0.14%·이더리움 1.06%↑...18시 00분 현재',
 '[클릭 신간] 힐빌리의 노래 외 2권',
 '"자리가 많지 않습니다"…31일 세종대서 은행 공동 취업설명회',
 '에임하이, 한류스타 이름 딴 가상화폐 내놓는다',
 '늘어나는 가상화폐 범죄',
 '[가상화폐 시황] 비트코인 -0.08%·이더리움 1.76%↑...19시 00분 현재',
 '[가상화폐 시황] 비트코인 0.00%·이더리움 -0.39%↓...20시 00분 현재']

In [33]:
df3 = df

In [37]:
df3

,date,time,title,content,press,link
0,2018.11.23,,비트코인 0.49%↑ 이더리움 0.84%↑ 리플 0.40%↑ 대시 0.89%↑ 에이...,,로이슈,http://www.lawissue.co.kr
1,2018.11.23,2018-11-23 00:01:10,"[가상화폐 뉴스] 11월 23일 00시 00분 비트코인(1.57%), 아이오타(7....",\n[한국경제TV 라이온봇 기자]\n\n\n\n[그림 1] 가상화폐 시세 (제공: ...,한국경제TV,https://n.news.naver.com/mnews/article/215/000...
2,2018.11.23,,비트코인 0.47%↑ 이더리움 0.67%↑ 리플 0.20%↑ 퀀텀 1.83%↑ 라이...,,로이슈,http://www.lawissue.co.kr
3,2018.11.23,,비트코인 0.45%↑ 이더리움 0.67%↑ 리플 0.40%↑ 라이트코인 0.78%↑...,,로이슈,http://www.lawissue.co.kr
4,2018.11.23,,[암호화폐 시황] 비트코인 0.47%↑·이더리움 0.67%↑...0시 00분 현재,,글로벌경제,http://www.getnews.co.kr
...,...,...,...,...,...,...
2923,2018.11.30,2018-11-30 23:31:17,"[가상화폐 뉴스] 11월 30일 23시 30분 비트코인(-6.28%), 이오스(-1...",\n[한국경제TV 라이온봇 기자]\n\n\n\n[그림 1] 가상화폐 시세 (제공: ...,한국경제TV,https://n.news.naver.com/mnews/article/215/000...
2924,2018.11.30,,[암호화폐 시황] 비트코인 0.51%↑ 비트코인캐시 1.04%↑ 비트코인골드 0% ...,,빅데이터뉴스,http://www.thebigdata.co.kr/
2925,2018.11.30,,"[세계 거래소별 가상화폐 시황] 30일, 23시 30분 현재",,로이슈,http://www.lawissue.co.kr
2926,2018.11.30,,"[암호화폐 시황] 코인제스트, 23시 40분 , 비트코인(0.53%), 이더리움(0...",,로이슈,http://www.lawissue.co.kr


In [64]:
len(df1)
len(df2)
len(df3)
print(len(df1)+len(df2)+len(df3))

9672


In [38]:
result = pd.concat([df1, df2, df3], ignore_index=True)
result.drop_duplicates(keep='first', inplace=True, ignore_index=True)
result

,date,time,title,content,press,link
0,2018.11.01,2018-11-01 00:00:59,"[가상화폐 뉴스] 11월 01일 00시 00분 비트코인(-0.7%), 스트리머(2....",\n[한국경제TV 라이온봇 기자]\n\n\n\n[그림 1] 가상화폐 시세 (제공: ...,한국경제TV,https://n.news.naver.com/mnews/article/215/000...
1,2018.11.01,,비트코인 -0.39%↓ 이더리움 -0.41%↓ 리플 -1.00%↓ 퀀텀 -0.71%...,,로이슈,http://www.lawissue.co.kr
2,2018.11.01,,비트코인 -0.36%↓ 이더리움 -0.72%↓ 리플 -0.60%↓ 라이트코인 -0....,,로이슈,http://www.lawissue.co.kr
3,2018.11.01,,비트코인 -0.45%↓ 이더리움 -0.75%↓ 리플 -1.00%↓ 대시 -1.43%...,,로이슈,http://www.lawissue.co.kr
4,2018.11.01,,"가상화폐(암호화폐) 00시 현재 시세, 이더리움 219,800원, 비트코인 7,09...",,내외경제tv,http://www.nbntv.co.kr
...,...,...,...,...,...,...
10452,2018.11.30,2018-11-30 23:31:17,"[가상화폐 뉴스] 11월 30일 23시 30분 비트코인(-6.28%), 이오스(-1...",\n[한국경제TV 라이온봇 기자]\n\n\n\n[그림 1] 가상화폐 시세 (제공: ...,한국경제TV,https://n.news.naver.com/mnews/article/215/000...
10453,2018.11.30,,[암호화폐 시황] 비트코인 0.51%↑ 비트코인캐시 1.04%↑ 비트코인골드 0% ...,,빅데이터뉴스,http://www.thebigdata.co.kr/
10454,2018.11.30,,"[세계 거래소별 가상화폐 시황] 30일, 23시 30분 현재",,로이슈,http://www.lawissue.co.kr
10455,2018.11.30,,"[암호화폐 시황] 코인제스트, 23시 40분 , 비트코인(0.53%), 이더리움(0...",,로이슈,http://www.lawissue.co.kr


In [39]:
result.to_csv('비트코인_20181101_20181130.csv', encoding='utf-8-sig', index=False)